In [ ]:
from __future__ import print_function, division

import torch
import torch.utils as utils
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import pandas as pd
import torchvision
from tqdm import tqdm
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor, ToPILImage
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 인플루언서 스타일 예측 및 csv에 적절히 붙이기

In [ ]:
# class names 리스트 불러오기
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/class_names.pkl","rb") as f:
    class_names = pickle.load(f)

In [ ]:
# 모델 불러오기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/model/model_class23.pt', map_location=device)
model_pr = model.eval() # eval

In [ ]:
# 인플루언서 csv 파일 불러오기
inf_post_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/inf_post_info2.csv')

In [ ]:
inf_post_info.head()

,user_name,post_id,text,timestamp,like,video
0,0nefence,3008175603962386523,어제 업로드 하려고 했어요 진짜에요,2023-01-04 8:54:58,526,0
1,0nefence,3006778853318091854,옷장에 옷은 많은데 당장 내일은 뭐 입어야 될지 모르겠어요 • •? 🤔,2023-01-02 10:39:52,-1,0
2,0nefence,3006161204775157958,#협찬 @myfipn_official \n\n다들 새해 첫날 어떻게 보내셨나요?\n...,2023-01-01 14:12:43,521,0
3,0nefence,3005347722609627475,마지막 인사 올립니다 꾸벅,2022-12-31 11:16:28,532,0
4,0nefence,3004577405444400836,한살 먹는 건 모르겠고 빨리 쉬고싶어요 🥹,2022-12-30 9:45:59,-1,0


In [ ]:
# 인플루언서 id 리스트로 저장
inf_dir = '/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/인플루언서정보_샘플1.zip (Unzipped Files)'
inf_ids = os.listdir(inf_dir)

In [ ]:
tf_toTensor = ToTensor() 

In [ ]:
# 스타일 분류 모델을 사용하여 모든 게시물의 이미지의 스타일을 분류
pred_idxs = []
pred_styles = []

for i in tqdm(range(len(inf_post_info))):
    
    inf_id = '@'+inf_post_info['user_name'][i]
    img_name = str(inf_post_info['post_id'][i])+'.jpg'
    
    try:
        img = Image.open(inf_dir+'/'+inf_id+'/images/'+img_name)
        img = img.resize((224, 224))

        # PIL to Tensor
        tensor_img = tf_toTensor(img)

        tensor_img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(tensor_img)
        tensor_img = torch.reshape(tensor_img, [-1, 3, 224, 224])

        pred_prob = model_pr(tensor_img) # 게시물 이미지를 모델로 input하여 predict probability을 얻음
        pred_idx = pred_prob.argmax().item() # 가장 probability가 높은 index를 추출
        pred_style = class_names[pred_idx] # 가장 probability가 높은 클래스를 추출

        pred_idxs.append(pred_idx)
        pred_styles.append(pred_style)

    except:
        pred_idxs.append(-1)
        pred_styles.append('no image') 

100%|██████████| 2989/2989 [26:20<00:00,  1.89it/s]


In [ ]:
print('pred_styles:', len(pred_styles))
print('inf_post_info:', len(inf_post_info))

pred_styles: 2989
inf_post_info: 2989


In [ ]:
# 예측된 style 추가
inf_post_info['post_style_idx'] = pred_idxs
inf_post_info['post_style'] = pred_styles

In [ ]:
inf_post_info.to_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/inf_pred_styles.csv', index = False)

# 제조사추천용 테이블

In [ ]:
# 인플루언서 게시글 스타일 분류 테이블 불러오기
inf_pred_styles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/inf_pred_styles.csv')

In [ ]:
inf_pred_styles.head()

,user_name,post_id,text,timestamp,like,video,post_style_idx,post_style
0,0nefence,3008175603962386523,어제 업로드 하려고 했어요 진짜에요,2023-01-04 8:54:58,526,0,0,기타
1,0nefence,3006778853318091854,옷장에 옷은 많은데 당장 내일은 뭐 입어야 될지 모르겠어요 • •? 🤔,2023-01-02 10:39:52,-1,0,10,스포티
2,0nefence,3006161204775157958,#협찬 @myfipn_official \n\n다들 새해 첫날 어떻게 보내셨나요?\n...,2023-01-01 14:12:43,521,0,0,기타
3,0nefence,3005347722609627475,마지막 인사 올립니다 꾸벅,2022-12-31 11:16:28,532,0,9,스트리트
4,0nefence,3004577405444400836,한살 먹는 건 모르겠고 빨리 쉬고싶어요 🥹,2022-12-30 9:45:59,-1,0,0,기타


In [ ]:
# 인플루언서 정보 테이블 불러오기
inf_ID_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/inf_ID_list.csv')
inf_ID_list.columns = ['user_name', 'post_cnt', 'follower', 'follow', 'gender', 'size']
inf_ID_list['user_name'] = inf_ID_list['user_name'].str.strip()

inf_ID_list['inf_style'] = 0
inf_ID_list['likes'] = 0
inf_ID_list['post_id'] = 0

In [ ]:
inf_ID_list.head()

,user_name,post_cnt,follower,follow,gender,size,inf_style,likes,post_id
0,@by_he.nique,3439,10만,1,여,여S,0,0,0
1,@c_eunnnnnnn,145,8.5만,642,여,여S,0,0,0
2,@oytoyt_,823,8032,2974,남,남M,0,0,0
3,@so_j2,96,3.6만,3.6만,여,여M,0,0,0
4,@eungil_j,312,6834,818,여,여L,0,0,0


In [ ]:
# 인플루언서 id 리스트로 저장
inf_dir = '/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/인플루언서정보_샘플1.zip (Unzipped Files)'
inf_ids = os.listdir(inf_dir)

In [ ]:
# 제조사 추천용 테이블 만들기
recomm_inf_item = inf_ID_list.copy()

for id in tqdm(inf_ids):
    inf_df = inf_pred_styles[inf_pred_styles['user_name']==id[1:]]
    inf_style_df = inf_df.groupby(['post_style']).agg({'like':'sum'}).sort_values('like', ascending = False).reset_index()

    inf_style = inf_style_df['post_style'][0] # 좋아요 개수가 가장 많은 스타일 -> 해당 인플루언서의 스타일
    if inf_style == 'no image' and len(inf_style_df) >= 2:
        inf_style = inf_style_df['post_style'][1] # 스타일이 no image로 분류되었을 경우, 두번째로 좋아요 개수가 많은 스타일을 선택

    inf_style_likes = inf_style_df['like'][0] # 스타일에 해당하는 게시물의 좋아요 수의 총합: 해당 스타일에 대한 인플루언서의 영향력

    # 스타일에 해당하는 게시물 중 좋아요 개수가 가장 많은 피드의 post id
    inf_style_post = inf_df[inf_df['post_style']==inf_style].sort_values('like', ascending = False)['post_id'].iloc[0] 
    
    idx = recomm_inf_item[recomm_inf_item['user_name']==id].index.item()
    recomm_inf_item.iloc[idx, -3] = inf_style # 좋아요 개수가 가장 많은 스타일을 데이터프레임에 입력
    recomm_inf_item.iloc[idx, -2] = inf_style_likes # 해당 스타일에 대한 총 좋아요 개수를 데이터프레임에 입력
    recomm_inf_item.iloc[idx, -1] = inf_style_post # 해당 스타일로 분류된 게시글 중 가장 좋아요 수가 많은 post id 입력
    

100%|██████████| 100/100 [00:00<00:00, 165.21it/s]


In [ ]:
recomm_inf_item.to_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/recomm_inf_item.csv', index = False)

# 소비자추천용 테이블

In [ ]:
# 인플루언서 게시글 스타일 분류 테이블 불러오기
inf_pred_styles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/inf_pred_styles.csv')

In [ ]:
inf_pred_styles.head()

,user_name,post_id,text,timestamp,like,video,post_style_idx,post_style
0,0nefence,3008175603962386523,어제 업로드 하려고 했어요 진짜에요,2023-01-04 8:54:58,526,0,0,기타
1,0nefence,3006778853318091854,옷장에 옷은 많은데 당장 내일은 뭐 입어야 될지 모르겠어요 • •? 🤔,2023-01-02 10:39:52,-1,0,10,스포티
2,0nefence,3006161204775157958,#협찬 @myfipn_official \n\n다들 새해 첫날 어떻게 보내셨나요?\n...,2023-01-01 14:12:43,521,0,0,기타
3,0nefence,3005347722609627475,마지막 인사 올립니다 꾸벅,2022-12-31 11:16:28,532,0,9,스트리트
4,0nefence,3004577405444400836,한살 먹는 건 모르겠고 빨리 쉬고싶어요 🥹,2022-12-30 9:45:59,-1,0,0,기타


In [ ]:
# 인플루언서 정보 테이블 불러오기
inf_ID_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/inf_ID_list.csv')
inf_ID_list.columns = ['user_name', 'post_cnt', 'follower', 'follow', 'gender', 'size']
inf_ID_list['user_name'] = inf_ID_list['user_name'].str.strip()

inf_ID_list['inf_style'] = 0
inf_ID_list['count'] = 0
inf_ID_list['post_id'] = 0

In [ ]:
inf_ID_list.head()

,user_name,post_cnt,follower,follow,gender,size,inf_style,count,post_id
0,@by_he.nique,3439,10만,1,여,여S,0,0,0
1,@c_eunnnnnnn,145,8.5만,642,여,여S,0,0,0
2,@oytoyt_,823,8032,2974,남,남M,0,0,0
3,@so_j2,96,3.6만,3.6만,여,여M,0,0,0
4,@eungil_j,312,6834,818,여,여L,0,0,0


In [ ]:
# 인플루언서 id 리스트로 저장
inf_dir = '/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/인플루언서정보_샘플1.zip (Unzipped Files)'
inf_ids = os.listdir(inf_dir)

In [ ]:
# 소비자 추천용 테이블 만들기
recomm_inf_cust = inf_ID_list.copy()

for id in tqdm(inf_ids):
    inf_df = inf_pred_styles[inf_pred_styles['user_name']==id[1:]]
    inf_style_df = inf_df.groupby(['post_style']).agg({'like':'count'}).sort_values('like', ascending = False).reset_index()

    inf_style = inf_style_df['post_style'][0] # 피드 개수가 가장 많은 스타일 -> 해당 인플루언서의 스타일
    if inf_style == 'no image' and len(inf_style_df) >= 2:
        inf_style = inf_style_df['post_style'][1] # 스타일이 no image로 분류되었을 경우, 두번째로 좋아요 개수가 많은 스타일을 선택
        
    inf_style_cnt = inf_style_df['like'][0] # 스타일에 해당하는 피드 개수: 해당 스타일에 대한 인플루언서의 관심

    # 스타일에 해당하는 게시물 중 좋아요 개수가 가장 많은 피드의 post id
    inf_style_post = inf_df[inf_df['post_style']==inf_style].sort_values('like', ascending = False)['post_id'].iloc[0] 
    
    idx = recomm_inf_cust[recomm_inf_cust['user_name']==id].index.item()
    recomm_inf_cust.iloc[idx, -3] = inf_style # 게시물 수가 가장 많은 스타일을 데이터프레임에 입력
    recomm_inf_cust.iloc[idx, -2] = inf_style_cnt # 해당 스타일에 대한 게시물 개수를 데이터프레임에 입력
    recomm_inf_cust.iloc[idx, -1] = inf_style_post # 해당 스타일로 분류된 게시글 중 가장 좋아요 수가 많은 post id 입력
    

100%|██████████| 100/100 [00:00<00:00, 200.91it/s]


In [ ]:
recomm_inf_cust.to_csv('/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/recomm_inf_cust.csv', index = False)

# 프로토타입에 적용할 코드

In [ ]:
# class names 리스트 불러오기
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/KUIAI/KUIAI/sample_data/class_names.pkl","rb") as f:
    class_names = pickle.load(f)

In [ ]:
tf_toTensor = ToTensor() 

In [ ]:
# 스타일 분류 모델 사용 함수
def predict_style(INPUT, MODEL_PATH, device, class_names):
    """
    
    INPUT: input image
    MODEL_PATH: model 경로
    device: cpu
    class_names: 스타일 종류

    """

    img = Image.open(INPUT)
    img = img.resize((224, 224))
    tensor_img = tf_toTensor(img)
    tensor_img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(tensor_img)
    tensor_img = torch.reshape(tensor_img, [-1, 3, 224, 224])

    model = torch.load(MODEL_PATH, map_location=device)
    model_pr = model.eval()

    pred = model_pr(tensor_img).argmax()
    pred_style = class_names[pred]

    return pred_style

In [ ]:
# 제조사 추천시스템 사용 함수
def recomm_inf_item(CSV_PATH, gender, size, style):
    """
    
    CSV_PATH: 인플루언서 정보 csv 파일 불러오기(제조사추천용)
    gender: 사용자가 원하는 인플루언서의 성별
    size: 사용자가 원하는 인플루언서의 체구
    style: predict_style 함수에서 predict된 input image의 스타일

    """

    # 제조사추천용 인플루언서 csv 파일 불러오기
    inf_post_info = pd.read_csv(CSV_PATH)

    # 사용자가 입력한 정보대로 인플루언서 필터링
    inf_post_info = inf_post_info[inf_post_info['gender'] == gender]
    inf_post_info = inf_post_info[inf_post_info['size'] == size]
    # 사용자가 입력한 이미지의 style(모델로 예측됨)로 인플루언서 필터링
    inf_post_info = inf_post_info[inf_post_info['inf_style'] == style]

    # 좋아요 수로 인플루언서 정렬
    inf_post_info = inf_post_info.sort_values('likes', ascending = False) # 제조사가 원하는 스타일에 대해 영향력이 큰 인플루언서 순서대로 나타남

    recomm_inf_ids = inf_post_info['user_name'][:3].to_list()
    recomm_inf_imgs = inf_post_info['post_id'][:3].to_list()

    return recomm_inf_ids, recomm_inf_imgs

In [ ]:
# 소비자 추천시스템 사용 함수
def recomm_inf_cust(CSV_PATH, gender, size, style):
    """
    
    CSV_PATH: 인플루언서 정보 csv 파일 불러오기(소비자추천용)
    gender: 사용자가 원하는 인플루언서의 성별
    size: 사용자가 원하는 인플루언서의 체구
    style: predict_style 함수에서 predict된 input image의 스타일

    """

    # 소비자추천용 인플루언서 csv 파일 불러오기
    inf_post_info = pd.read_csv(CSV_PATH)

    # 사용자가 입력한 정보대로 인플루언서 필터링
    inf_post_info = inf_post_info[inf_post_info['gender'] == gender]
    inf_post_info = inf_post_info[inf_post_info['size'] == size]
    # 사용자가 입력한 이미지의 style(모델로 예측됨)로 인플루언서 필터링
    inf_post_info = inf_post_info[inf_post_info['inf_style'] == style]

    # 스타일 등장 피드 개수로 인플루언서 정렬
    inf_post_info = inf_post_info.sort_values('count', ascending = False) # 소비자가 원하는 피드가 많은 인플루언서 순서대로 나타남

    recomm_inf_ids = inf_post_info['user_name'][:3].to_list()
    recomm_inf_imgs = inf_post_info['post_id'][:3].to_list()

    return recomm_inf_ids, recomm_inf_imgs